In [ ]:
import asyncio
import nest_asyncio
import concurrent.futures.process
import numpy as np
from Workspace.BackTest.DataFactory import FactoryManager
import SystemConfig
from typing import List, Dict, Any


nest_asyncio.apply()

start_date = "2025-01-01"
end_date = "2025-01-5"
symbol = SystemConfig.Streaming.symbols
intervals = SystemConfig.Streaming.intervals + ["1m"]

ins_factory = FactoryManager(start_date, end_date)

symbols = ["BTCUSDT"]#, "ETHUSDT", "ADAUSDT"]
data = asyncio.run(ins_factory.fetch_multiple_klines(symbols, intervals))

process_data = ins_factory.process_for_analysis(data)

  📨 BTCUSDT_5m 수신 완료
  📨 BTCUSDT_3m 수신 완료
  📨 BTCUSDT_1m 수신 완료


In [ ]:
import Workspace.Utils.BaseUtils as base_utils

def generate_kline_closing_sync_2(base_data:np.ndarray, selec_data:np.ndarray, interval:str):
    result = {}
    timestamp_range = base_utils.get_interval_ms_seconds(interval) - 1
    temp_data = []
    
    
    for (open_timestamp, open_price, high_price, low_price, close_price, 
     volume, close_timestamp, volume_total_usdt, trades_count, 
     taker_asset_volume, taker_quote_volume) in zip(*base_data.T):
        # open_timestamp = 시작 타임스템프
        # open_price = 시작 가격
        # high_price = 최고 가격
        # low_price = 최저 가격
        # close_price = 마지막 가격
        # volume = 거래량(단위 : coin)
        # close_timestamp = 종료 타임스템프
        # volume_total_usdt = 거래량(단위 : usdt)
        # trades_count = 총 거래횟수
        # taker_asset_volume =  시장가 주문 거래량(단위 : coin)
        # taker_quote_volume = 시장가 주문 거래량(단위 : usdt)

        condition = np.where(
            (selec_data[:, 0] <= open_timestamp)
            & (selec_data[:, 6] >= close_timestamp))

        target_data = selec_data[condition]
        
        target_open_timestamp = target_data[0, 0]  # 단일값이 확실할 경우
        target_close_timestamp = target_data[0, 6]  # 단일값이 확실할 경우

        new_data_condition = np.where(
            (base_data[:, 0] >= target_open_timestamp)
            & (base_data[:, 6] <= close_timestamp)
        )  # close_timestamp는 현재 data종료 시간 기준으로 해야한다.

        new_base_data = base_data[new_data_condition]

        timestamp_diff = new_base_data[-1, 6] - new_base_data[0, 0]
        if timestamp_range == timestamp_diff:
            new_data = target_data[0]
        else:
            new_data = [
                target_open_timestamp,
                new_base_data[0, 1],
                np.max(new_base_data[:, 2]),
                np.min(new_base_data[:, 3]),
                new_base_data[-1, 4],
                np.sum(new_base_data[:, 5]),
                target_close_timestamp,
                np.sum(new_base_data[:, 7]),
                np.sum(new_base_data[:, 8]),
                np.sum(new_base_data[:, 9]),
                np.sum(new_base_data[:, 10]),
                0,
            ]
        temp_data.append(new_data)
    result[interval] = np.array(temp_data, float)
    
    return result

In [23]:
base = process_data["BTCUSDT"]["1m"]
target = process_data["BTCUSDT"]["3m"]

In [28]:
target

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.73568960e+12, 9.35488000e+04, 9.36900000e+04, ...,
        1.20141000e+02, 1.12466529e+07, 0.00000000e+00],
       [1.73568978e+12, 9.36885000e+04, 9.36885000e+04, ...,
        4.47990000e+01, 4.19559992e+06, 0.00000000e+00],
       ...,
       [1.73603466e+12, 9.82550000e+04, 9.82551000e+04, ...,
        1.08860000e+01, 1.06945263e+06, 0.00000000e+00],
       [1.73603484e+12, 9.82204000e+04, 9.82278000e+04, ...,
        2.97120000e+01, 2.91821296e+06, 0.00000000e+00],
       [1.73603502e+12, 9.82067000e+04, 9.82068000e+04, ...,
        2.42400000e+01, 2.37975032e+06, 0.00000000e+00]], shape=(1921, 12))

In [29]:
dummy = [1,2,3,4,5]
a, b, c, d, e = dummy

In [30]:
a

1

In [31]:
b

2

In [32]:
c

3

In [33]:
d

4

In [ ]:
E